In [1]:
import requests
from lxml import etree
import re
import pandas as pd
import time
from bs4 import BeautifulSoup

In [4]:
ssrequest = requests.Session()
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36'
                         ' (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'
            }
response = ssrequest.get("https://www.scotchwhiskyauctions.com/auctions/",headers =headers)
# jd_cookies_dict


HTML=etree.HTML(response.text)

In [5]:
Auction_List = HTML.xpath('//*[@id="contentstart"]/div[2]/div/a')
action_list = []
for auction in Auction_List:
    auction_title = auction.xpath('./span/span/text()')[0]
    auction_date = auction.xpath('./span/span/text()')[1]
    auction_products = auction.xpath('./span/span/text()')[2]
    auction_link ='https://www.scotchwhiskyauctions.com/'+ auction.xpath('./@href')[0]
    auction_dic = {'auction_title':auction_title,'auction_date':auction_date,'auction_products':auction_products,'auction_link':auction_link}
    action_list.append(auction_dic)

In [6]:
auction_df = pd.DataFrame(action_list)
auction_df

,auction_title,auction_date,auction_products,auction_link
0,The 113th Auction,"Ends on November 1, 2020",There are 8152 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
1,The 112th Auction,"Ended on October 4, 2020",There are 7638 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
2,The 111th Auction,"Ended on September 6, 2020",There are 7343 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
3,The 110th Auction,"Ended on August 2, 2020",There are 6759 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
4,The 109th Auction,"Ended on July 5, 2020",There are 5640 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
...,...,...,...,...
110,The Fifth Auction,"Ended on August 29, 2011",There are 161 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
111,The Fourth Auction,"Ended on August 1, 2011",There are 132 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
112,The Third Auction,"Ended on July 4, 2011",There are 78 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...
113,The Second Auction,"Ended on June 5, 2011",There are 94 lots in this auction.,https://www.scotchwhiskyauctions.com/auctions/...


In [7]:
def get_auction_product_data(auction_link,auction_product_num):
    pro_num_per_page = 5000
    total_page_num = int(int(auction_product_num)/pro_num_per_page)+2
    products_list = []
    for page_num in range(1,total_page_num):
        url_mod = '?page={0}&area=&sort=price&order=desc&perpage={1}'.format(page_num,pro_num_per_page)
        auction_page_link = auction_link+url_mod
        print(auction_page_link)
        response1 =  ssrequest.get(auction_page_link,headers =headers)
        time.sleep(8)
        soup = BeautifulSoup(response1.text)

        for product in soup.select('.productsubcats.products > a'):
            product_title = product.select('.prodtitle')[0].string
            product_lot = product.select('.prodlot.priceline.curprice')[0].string
            product_lot = product_lot.split(' ')[2]
            try:
                product_price = product.select('.price')[0].string
            except IndexError:
                product_price = 'Item failed to meet reserve.'
            product_link = 'https://www.'+ product['href']
            product_dic = {'product_title':product_title,'product_lot':product_lot,'product_price':product_price,'product_link':product_link}
            products_list.append(product_dic)

    return products_list

In [8]:
fist_to_csv = 1

In [14]:
for auction in auction_df.values[100:]:
    auction_title = auction[0]
    auction_date = auction[1]
    auction_product_num = auction[2].split(' ')[2]
    auction_link = auction[3]
    product_list = get_auction_product_data(auction_link,auction_product_num)
    products_df = pd.DataFrame(product_list)
    auction_title = auction_title.split(' ')[1]
    products_df['auction_No'] = auction_title
    
    auction_date = auction_date.split(' ')[2:]
    products_df['auction_end_date'] = ' '.join(auction_date)
    
    products_df['auction_link'] = auction_link
    if fist_to_csv == 1:
        products_df.to_csv('product_data.csv',header =True,index =False)
        fist_to_csv = 0 
    else:
        products_df.to_csv('product_data.csv',header =False,index =False,mode = 'a')
        print('存入成功')

https://www.scotchwhiskyauctions.com/auctions/43_the-15th-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/40_the-14th-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/39_the-13th-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/38_the-12th-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/37_the-11th-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/34_the-10th-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/33_the-ninth-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/auctions/31_the-eighth-auction/?page=1&area=&sort=price&order=desc&perpage=5000
存入成功
https://www.scotchwhiskyauctions.com/

In [91]:
products_df

,product_title,product_lot,product_price,product_link,auction_No,auction_end_date,auction_link
0,Karuizawa 1983 Nepal Appeal Cask #3557,0552624,£9000.00,https://www.auctions/93_the-55th-auction/80244...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
1,Laphroaig 1960 Vintage Reserve,0550806,£4000.00,https://www.auctions/93_the-55th-auction/78372...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
2,Karuizawa 30 Year Old Cask #3619 For Kinlonz C...,0552100,£3600.00,https://www.auctions/93_the-55th-auction/79719...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
3,Karuizawa 30 Year Old Samurai Cask #3622,0550360,£3300.00,https://www.auctions/93_the-55th-auction/77828...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
4,Talisker 1973 28 Year Old,0550811,£3200.00,https://www.auctions/93_the-55th-auction/78377...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
...,...,...,...,...,...,...,...
5663,Highland Fusilier 8 Year Old Pure Malt G&M Min...,0550706,£0.00,https://www.auctions/93_the-55th-auction/78263...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
5664,Highland Fusilier 12 Year Old G&M Mini 5cl,0550707,£0.00,https://www.auctions/93_the-55th-auction/78264...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
5665,Glenturret 8 Year Old Mini 5cl,0550709,£0.00,https://www.auctions/93_the-55th-auction/78266...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...
5666,Glenturret 1972 Mini 5cl,0550710,£0.00,https://www.auctions/93_the-55th-auction/78267...,111th,"September 6, 2020",https://www.scotchwhiskyauctions.com/auctions/...


In [175]:
products_df[products_df['product_title'] == 'Macallan 1950 Exceptional Cask #13 2018 Release']

,product_title,product_lot,product_price,product_link
7588,Macallan 1950 Exceptional Cask #13 2018 Release,Lot number: 112-07711,Item failed to meet reserve.,https://www.auctions/153_the-112th-auction/414...


16:Single Malt Whisky
17:Blended Whisky
18:Grain Whisky
35:Rum
42:Brandy / Cognac 
28:Other Spirits
32:Miniature
33:lost Distilleries
34:Independent Bottlers

Region
3:Campbeltown
4:Highland
5:Island
6:Islay
7:Lowland
8:Speyside
9:Irish
12:Japanese
14:USA
15:World Whisky
31:Taiwanese

Cask Type/Finish
37:Sherry
38:Bourbon
39:Virgin Oak
40:Rum
41:Still Wine
43:Port
44:Madeira
46:Beer
47:American White Oak
48:French Oak
45:Mizunara
49:Sauternes
51:Marsala
52:Other Fortified Wine
53:Cognac / Armagnac / Brandy
55:Calvados / Apple Brandy
57:Other

